In [86]:
# project_id = 'Enter Project ID' 
# location = 'us' 
# processor_id = 'Enter Processor ID' 
processor_version = 'rc' 
# file_path = "filepath.pdf" 
# mime_type = 'application/pdf'

GCP_PROJECT= PROJECT_ID=project_id='my-project-0004-346516'
LOCATION = location = "us"  # Format is 'us' or 'eu'
PROCESSOR_ID = processor_id = "c66ce4457bf5af37"  # Create processor in Cloud Console
FILE_PATH =file_path =  "Winnie_the_Pooh_3_Pages.pdf"
# Refer to https://cloud.google.com/document-ai/docs/file-types
# for supported file types
MIME_TYPE =mime_type = "application/pdf"

"gs://my-project-0004-346516/matchingengine"

'gs://my-project-0004-346516/matchingengine'

In [39]:
%pip install --upgrade --quiet  langchain-google-genai



Note: you may need to restart the kernel to use updated packages.


In [40]:
def online_process(
    project_id: str,
    location: str,
    processor_id: str,
    file_path: str,
    mime_type: str,
) -> documentai.Document:
    """
    A function to process a document online using Google Document AI.
    """

    # Define an options dictionary, which includes the API's URL. This is used to connect to Google's Document AI service
    opts = {"api_endpoint": f"{location}-documentai.googleapis.com"}

    # Create a Document AI client, think of it as our bridge for communicating with Google's services
    documentai_client = documentai.DocumentProcessorServiceClient(client_options=opts)

    # Generate the complete name of the processor
    # You need to first create a processor in the Google Cloud console
    resource_name = documentai_client.processor_path(project_id, location, processor_id)

    # Read in the document you want to analyze (like an image or PDF), and store it in the variable image_content
    with open(file_path, "rb") as image:
        image_content = image.read()

        # Convert the read document into a format that Google Document AI can understand, i.e., a RawDocument object
        raw_document = documentai.RawDocument(
            content=image_content, mime_type=mime_type
        )
        # Create a request, which includes the name of the processor and the document we want to analyze
        request = documentai.ProcessRequest(
            name=resource_name, raw_document=raw_document
        )
        # Send our request and receive the analysis results
        result = documentai_client.process_document(request=request)
        
        print("Document processing complete.")
        # print(f"Text: {document_object.text}")
        
        # Return this analysis result
        return result.document
    
    

# Load Binary Data into Document AI RawDocument Object
# raw_document = documentai.RawDocument(content=image_content, mime_type=MIME_TYPE)

# Configure the process request
# request = documentai.ProcessRequest(name=RESOURCE_NAME, raw_document=raw_document)

# # Use the Document AI client to process the sample form
# result = docai_client.process_document(request=request)

# document_object = result.document
# print("Document processing complete.")
# print(f"Text: {document_object.text}")

# page_text =document_object.text

In [41]:
def trim_text(text: str): 
    """ Removes spaces and newline characters. """ 
    return text.strip().replace("\n", " ")

### https://cloud.google.com/document-ai/docs/handle-response

In [42]:
from typing import Optional, Sequence

from google.api_core.client_options import ClientOptions
from google.cloud import documentai

def print_page_dimensions(dimension: documentai.Document.Page.Dimension) -> None:
    print(f"    Width: {str(dimension.width)}")
    print(f"    Height: {str(dimension.height)}")


def print_detected_langauges(
    detected_languages: Sequence[documentai.Document.Page.DetectedLanguage],
) -> None:
    print("    Detected languages:")
    for lang in detected_languages:
        print(f"        {lang.language_code} ({lang.confidence:.1%} confidence)")


def print_blocks(blocks: Sequence[documentai.Document.Page.Block], text: str) -> None:
    print(f"    {len(blocks)} blocks detected:")
    first_block_text = layout_to_text(blocks[0].layout, text)
    print(f"        First text block: {repr(first_block_text)}")
    last_block_text = layout_to_text(blocks[-1].layout, text)
    print(f"        Last text block: {repr(last_block_text)}")


def print_paragraphs(
    paragraphs: Sequence[documentai.Document.Page.Paragraph], text: str
) -> None:
    print(f"    {len(paragraphs)} paragraphs detected:")
    first_paragraph_text = layout_to_text(paragraphs[0].layout, text)
    print(f"        First paragraph text: {repr(first_paragraph_text)}")

    last_paragraph_text = layout_to_text(paragraphs[-1].layout, text)
    print(f"        Last paragraph text: {repr(last_paragraph_text)}")


def print_lines(lines: Sequence[documentai.Document.Page.Line], text: str) -> None:
    print(f"    {len(lines)} lines detected:")
    first_line_text = layout_to_text(lines[0].layout, text)
    print(f"        First line text: {repr(first_line_text)}")
    last_line_text = layout_to_text(lines[-1].layout, text)
    print(f"        Last line text: {repr(last_line_text)}")


def print_tokens(tokens: Sequence[documentai.Document.Page.Token], text: str) -> None:
    print(f"    {len(tokens)} tokens detected:")
    first_token_text = layout_to_text(tokens[0].layout, text)
    first_token_break_type = tokens[0].detected_break.type_.name
    print(f"        First token text: {repr(first_token_text)}")
    print(f"        First token break type: {repr(first_token_break_type)}")
    if tokens[0].style_info:
        print_style_info(tokens[0].style_info)

    last_token_text = layout_to_text(tokens[-1].layout, text)
    last_token_break_type = tokens[-1].detected_break.type_.name
    print(f"        Last token text: {repr(last_token_text)}")
    print(f"        Last token break type: {repr(last_token_break_type)}")
    if tokens[-1].style_info:
        print_style_info(tokens[-1].style_info)


def print_symbols(
    symbols: Sequence[documentai.Document.Page.Symbol], text: str
) -> None:
    print(f"    {len(symbols)} symbols detected:")
    first_symbol_text = layout_to_text(symbols[0].layout, text)
    print(f"        First symbol text: {repr(first_symbol_text)}")
    last_symbol_text = layout_to_text(symbols[-1].layout, text)
    print(f"        Last symbol text: {repr(last_symbol_text)}")


def print_image_quality_scores(
    image_quality_scores: documentai.Document.Page.ImageQualityScores,
) -> None:
    print(f"    Quality score: {image_quality_scores.quality_score:.1%}")
    print("    Detected defects:")

    for detected_defect in image_quality_scores.detected_defects:
        print(f"        {detected_defect.type_}: {detected_defect.confidence:.1%}")


def print_style_info(style_info: documentai.Document.Page.Token.StyleInfo) -> None:
    """
    Only supported in version `pretrained-ocr-v2.0-2023-06-02`
    """
    print(f"           Font Size: {style_info.font_size}pt")
    print(f"           Font Type: {style_info.font_type}")
    print(f"           Bold: {style_info.bold}")
    print(f"           Italic: {style_info.italic}")
    print(f"           Underlined: {style_info.underlined}")
    print(f"           Handwritten: {style_info.handwritten}")
    print(
        f"           Text Color (RGBa): {style_info.text_color.red}, {style_info.text_color.green}, {style_info.text_color.blue}, {style_info.text_color.alpha}"
    )


def print_visual_elements(
    visual_elements: Sequence[documentai.Document.Page.VisualElement], text: str
) -> None:
    """
    Only supported in version `pretrained-ocr-v2.0-2023-06-02`
    """
    checkboxes = [x for x in visual_elements if "checkbox" in x.type]
    math_symbols = [x for x in visual_elements if x.type == "math_formula"]

    if checkboxes:
        print(f"    {len(checkboxes)} checkboxes detected:")
        print(f"        First checkbox: {repr(checkboxes[0].type)}")
        print(f"        Last checkbox: {repr(checkboxes[-1].type)}")

    if math_symbols:
        print(f"    {len(math_symbols)} math symbols detected:")
        first_math_symbol_text = layout_to_text(math_symbols[0].layout, text)
        print(f"        First math symbol: {repr(first_math_symbol_text)}")


def process_document(
    project_id: str,
    location: str,
    processor_id: str,
    processor_version: str,
    file_path: str,
    mime_type: str,
    process_options: Optional[documentai.ProcessOptions] = None,
) -> documentai.Document:
    # You must set the `api_endpoint` if you use a location other than "us".
    client = documentai.DocumentProcessorServiceClient(
        client_options=ClientOptions(
            api_endpoint=f"{location}-documentai.googleapis.com"
        )
    )

    # The full resource name of the processor version, e.g.:
    # `projects/{project_id}/locations/{location}/processors/{processor_id}/processorVersions/{processor_version_id}`
    # You must create a processor before running this sample.
    name = client.processor_version_path(
        project_id, location, processor_id, processor_version
    )

    # Read the file into memory
    with open(file_path, "rb") as image:
        image_content = image.read()

    # Configure the process request
    request = documentai.ProcessRequest(
        name=name,
        raw_document=documentai.RawDocument(content=image_content, mime_type=mime_type),
        # Only supported for Document OCR processor
        process_options=process_options,
    )

    result = client.process_document(request=request)

    # For a full list of `Document` object attributes, reference this page:
    # https://cloud.google.com/document-ai/docs/reference/rest/v1/Document
    return result.document


def layout_to_text(layout: documentai.Document.Page.Layout, text: str) -> str:
    """
    Document AI identifies text in different parts of the document by their
    offsets in the entirety of the document"s text. This function converts
    offsets to a string.
    """
    # If a text segment spans several lines, it will
    # be stored in different text segments.
    return "".join(
        text[int(segment.start_index) : int(segment.end_index)]
        for segment in layout.text_anchor.text_segments
    )


In [43]:
document = online_process(
    project_id=project_id,
    location=location,
    processor_id=processor_id,
    file_path=file_path,
    mime_type=mime_type,
)

names = []
name_confidence = []
values = []
value_confidence = []

for page in document.pages:
    print(page)
    # for field in page.form_fields:
    #     names.append(trim_text(field.field_name.text_anchor.content))
    #     name_confidence.append(field.field_name.confidence)
    #     values.append(trim_text(field.field_value.text_anchor.content))
    #     value_confidence.append(field.field_value.confidence)

# df = pd.DataFrame(
#     {
#         "Field Name": names,
#         "Field Name Confidence": name_confidence,
#         "Field Value": values,
#         "Field Value Confidence": value_confidence,
#     }
# )

IOPub data rate exceeded.
The Jupyter server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--ServerApp.iopub_data_rate_limit`.

Current values:
ServerApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
ServerApp.rate_limit_window=3.0 (secs)



In [44]:
print("all text", document.text)

all text IN WHICH We Are Introduced to
CHAPTER I
Winnie-the-Pooh and Some
Bees, and the Stories Begin
HERE is Edward Bear, coming
downstairs now, bump, bump, bump, on the back
of his head, behind Christopher Robin. It is, as far
as he knows, the only way of coming downstairs,
but sometimes he feels that there really is another
way, if only he could stop bumping for a moment
and think of it. And then he feels that perhaps there
isn't. Anyhow, here he is at the bottom, and ready
to be introduced to you. Winnie-the-Pooh.
When I first heard his name, I said, just as you
are going to say, "But I thought he was a boy?"
"So did I," said Christopher Robin.
"Then you can't call him Winnie?"
"I don't."
"But you said--"
"He's Winnie-ther-Pooh. Don't you know what
'ther' means?”
I
Digitized by
Google
WINNIE-THE-POOH
“Ah, yes, now I do," I said quickly; and I hope
you do too, because it is all the explanation you are
going to get.
Sometimes Winnie-the-Pooh likes a game of some
sort when he comes do

In [45]:
# for page in document.pages:
#     print(f"Page {page.page_number}:")
#     print_page_dimensions(page.dimension)
#     print_detected_langauges(page.detected_languages)

#     print_blocks(page.blocks, text)
#     print_paragraphs(page.paragraphs, text)
#     print_lines(page.lines, text)
#     print_tokens(page.tokens, text)
    
#     print(page.paragraphs.reverse)
    
# #     if page.symbols:
# #         print_symbols(page.symbols, text)

# #     if page.image_quality_scores:
# #         print_image_quality_scores(page.image_quality_scores)

# #     if page.visual_elements:
# #         print_visual_elements(page.visual_elements, text)

Page 1:
    Width: 1589.0
    Height: 2515.0
    Detected languages:
        en (98.0% confidence)


NameError: name 'text' is not defined

## Using langchain for Doc AI on GCP

In [46]:
%pip install --upgrade --quiet  google-cloud-documentai
%pip install --upgrade --quiet  google-cloud-documentai-toolbox

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [47]:
# GCS_OUTPUT_PATH = "gs://BUCKET_NAME/FOLDER_PATH"

GCS_OUTPUT_PATH= BUCKET_URI#"gs://tianhaoz-test/saurabh"

# PROCESSOR_NAME = "projects/cloud-llm-preview1/locations/us-central1/processors/96c7b8734e4ddaba"

# PROCESSOR_NAME = 'projects/PROJECT_ID/locations/us/processors/PROCESSOR_ID'

PROCESSOR_NAME = 'projects/255766800726/locations/us/processors/c66ce4457bf5af37'

# endpoint="projects/cloud-llm-preview1/locations/us-central1/publishers/google/models/medlm-large", instances=instances, parameters=parameters, safety_settings=[]


In [48]:
from langchain_community.document_loaders.blob_loaders import Blob
from langchain_community.document_loaders.parsers import DocAIParser

In [49]:
parser = DocAIParser(
    location="us", processor_name=PROCESSOR_NAME, gcs_output_path=GCS_OUTPUT_PATH
)

In [55]:
!gsutil cp ./Winnie_the_Pooh_3_Pages.pdf "$BUCKET_URI"

Copying file://./Winnie_the_Pooh_3_Pages.pdf [Content-Type=application/pdf]...
/ [1 files][531.1 KiB/531.1 KiB]                                                
Operation completed over 1 objects/531.1 KiB.                                    


In [52]:
BUCKET_URI

'gs://my-project-0004-346516/matchingengine'

In [53]:
blob = Blob(
    path="gs://my-project-0004-346516/matchingengine/Winnie_the_Pooh_3_Pages.pdf"
)

### This below would take a lot of time (about 2-3 for a small PDF)

In [56]:
docs = list(parser.lazy_parse(blob))
print(len(docs))



3


In [57]:
page_contents = [doc.page_content for doc in docs]


In [58]:
# docs[0].page_content
page_contents

['IN WHICH We Are Introduced to\nCHAPTER I\nWinnie-the-Pooh and Some\nBees, and the Stories Begin\nHERE is Edward Bear, coming\ndownstairs now, bump, bump, bump, on the back\nof his head, behind Christopher Robin. It is, as far\nas he knows, the only way of coming downstairs,\nbut sometimes he feels that there really is another\nway, if only he could stop bumping for a moment\nand think of it. And then he feels that perhaps there\nisn\'t. Anyhow, here he is at the bottom, and ready\nto be introduced to you. Winnie-the-Pooh.\nWhen I first heard his name, I said, just as you\nare going to say, "But I thought he was a boy?"\n"So did I," said Christopher Robin.\n"Then you can\'t call him Winnie?"\n"I don\'t."\n"But you said--"\n"He\'s Winnie-ther-Pooh. Don\'t you know what\n\'ther\' means?”\nI\nDigitized by\nGoogle\n',
 'WINNIE-THE-POOH\n“Ah, yes, now I do," I said quickly; and I hope\nyou do too, because it is all the explanation you are\ngoing to get.\nSometimes Winnie-the-Pooh likes a g

In [59]:
import pandas as pd

# Create a DataFrame from the splitted texts
df = pd.DataFrame({'pagewise_texts': page_contents})

# Add a row number column
df['page_id'] = df.index + 1
# df['pagewise_texts'] = df['pagewise_texts'].page_content


# Print the DataFrame
print(df)

                                      pagewise_texts  page_id
0  IN WHICH We Are Introduced to\nCHAPTER I\nWinn...        1
1  WINNIE-THE-POOH\n“Ah, yes, now I do," I said q...        2
2  WE ARE INTRODUCED\nAB SANDER\nRNIG\nALSO\n"Win...        3


In [60]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

chunk_size=100
chunk_overlap=20

text_splitter = RecursiveCharacterTextSplitter(
    # Set a really small chunk size, just to show.
    chunk_size=chunk_size,
    chunk_overlap=chunk_overlap,
    separators= ["/,", "##", ">", "We"],#'\n\n', '\n'],
    length_function=len,
    is_separator_regex=False,
)

# splitted_texts = text_splitter.create_documents([page_text])
# print("one \n",splitted_texts[0])
# print("two \n",splitted_texts[1])

def split_text_chunks(text, chunk_size):
    return [text[i:i+chunk_size] for i in range(0, len(text), chunk_size)]


In [61]:
splitted_texts = []

for index, row in df.iterrows():
    splitted_texts += text_splitter.create_documents(row['pagewise_texts'])
    splitted_texts2 = text_splitter.split_text(row['pagewise_texts'])
    print(splitted_texts2)

    # df['splitted_texts'] = splitted_texts2
    
    df['splitted_texts'] = df['pagewise_texts'].apply(lambda x: text_splitter.split_text(x))


['IN WHICH', 'We Are Introduced to\nCHAPTER I\nWinnie-the-Pooh and Some\nBees, and the Stories Begin\nHERE is Edward Bear, coming\ndownstairs now, bump, bump, bump, on the back\nof his head, behind Christopher Robin. It is, as far\nas he knows, the only way of coming downstairs,\nbut sometimes he feels that there really is another\nway, if only he could stop bumping for a moment\nand think of it. And then he feels that perhaps there\nisn\'t. Anyhow, here he is at the bottom, and ready\nto be introduced to you. Winnie-the-Pooh.\nWhen I first heard his name, I said, just as you\nare going to say, "But I thought he was a boy?"\n"So did I," said Christopher Robin.\n"Then you can\'t call him Winnie?"\n"I don\'t."\n"But you said--"\n"He\'s Winnie-ther-Pooh. Don\'t you know what\n\'ther\' means?”\nI\nDigitized by\nGoogle\n']
['WINNIE-THE-POOH\n“Ah, yes, now I do," I said quickly; and I hope\nyou do too, because it is all the explanation you are\ngoing to get.\nSometimes Winnie-the-Pooh likes 

In [62]:
df.head()

,pagewise_texts,page_id,splitted_texts
0,IN WHICH We Are Introduced to\nCHAPTER I\nWinn...,1,"[IN WHICH, We Are Introduced to\nCHAPTER I\nWi..."
1,"WINNIE-THE-POOH\n“Ah, yes, now I do,"" I said q...",2,"[WINNIE-THE-POOH\n“Ah, yes, now I do,"" I said ..."
2,"WE ARE INTRODUCED\nAB SANDER\nRNIG\nALSO\n""Win...",3,"[WE ARE INTRODUCED\nAB SANDER\nRNIG\nALSO\n""Wi..."


In [102]:
df_exploded = df.explode('splitted_texts')


In [103]:
df_exploded.head()

,pagewise_texts,page_id,splitted_texts
0,IN WHICH We Are Introduced to\nCHAPTER I\nWinn...,1,IN WHICH
0,IN WHICH We Are Introduced to\nCHAPTER I\nWinn...,1,We Are Introduced to\nCHAPTER I\nWinnie-the-Po...
1,"WINNIE-THE-POOH\n“Ah, yes, now I do,"" I said q...",2,"WINNIE-THE-POOH\n“Ah, yes, now I do,"" I said q..."
2,"WE ARE INTRODUCED\nAB SANDER\nRNIG\nALSO\n""Win...",3,"WE ARE INTRODUCED\nAB SANDER\nRNIG\nALSO\n""Win..."


In [104]:
len(splitted_texts)

1796

In [105]:
df_exploded['splitted_texts_chunks'] = df_exploded['splitted_texts'].apply(lambda x: split_text_chunks(x,chunk_size))


In [106]:
df_exploded_2 = df_exploded.explode('splitted_texts_chunks')


In [107]:
df_exploded_2.head(10)

,pagewise_texts,page_id,splitted_texts,splitted_texts_chunks
0,IN WHICH We Are Introduced to\nCHAPTER I\nWinn...,1,IN WHICH,IN WHICH
0,IN WHICH We Are Introduced to\nCHAPTER I\nWinn...,1,We Are Introduced to\nCHAPTER I\nWinnie-the-Po...,We Are Introduced to\nCHAPTER I\nWinnie-the-Po...
0,IN WHICH We Are Introduced to\nCHAPTER I\nWinn...,1,We Are Introduced to\nCHAPTER I\nWinnie-the-Po...,"ear, coming\ndownstairs now, bump, bump, bump,..."
0,IN WHICH We Are Introduced to\nCHAPTER I\nWinn...,1,We Are Introduced to\nCHAPTER I\nWinnie-the-Po...,"is, as far\nas he knows, the only way of comin..."
0,IN WHICH We Are Introduced to\nCHAPTER I\nWinn...,1,We Are Introduced to\nCHAPTER I\nWinnie-the-Po...,"is another\nway, if only he could stop bumping..."
0,IN WHICH We Are Introduced to\nCHAPTER I\nWinn...,1,We Are Introduced to\nCHAPTER I\nWinnie-the-Po...,"erhaps there\nisn't. Anyhow, here he is at the..."
0,IN WHICH We Are Introduced to\nCHAPTER I\nWinn...,1,We Are Introduced to\nCHAPTER I\nWinnie-the-Po...,"Pooh.\nWhen I first heard his name, I said, ju..."
0,IN WHICH We Are Introduced to\nCHAPTER I\nWinn...,1,We Are Introduced to\nCHAPTER I\nWinnie-the-Po...,"?""\n""So did I,"" said Christopher Robin.\n""Then..."
0,IN WHICH We Are Introduced to\nCHAPTER I\nWinn...,1,We Are Introduced to\nCHAPTER I\nWinnie-the-Po...,"\n""He's Winnie-ther-Pooh. Don't you know what\..."
1,"WINNIE-THE-POOH\n“Ah, yes, now I do,"" I said q...",2,"WINNIE-THE-POOH\n“Ah, yes, now I do,"" I said q...","WINNIE-THE-POOH\n“Ah, yes, now I do,"" I said q..."


In [108]:
x=get_embeddings_wrapper(list(df_exploded_2.splitted_texts_chunks))
len(x)

100%|██████████| 5/5 [00:05<00:00,  1.09s/it]


768

In [109]:
from langchain_google_vertexai import VertexAIEmbeddings

embeddings = VertexAIEmbeddings(model="models/embedding-003")

text = "This is a test document."

query_result = embeddings.embed_query(text)
# print(query_result)

Model_name will become a required arg for VertexAIEmbeddings starting from Feb-01-2024. Currently the default is set to textembedding-gecko@001


In [110]:
df_exploded_2a = df_exploded_2.reindex()
df_exploded_2a = df_exploded_2a.reset_index()

In [111]:
# df = pd.DataFrame({"id": [0], "embedding": [[0.03090338036417961, -0.0172298401594162]]})


In [112]:
# x=embeddings.embed_query(df_exploded_2a.loc[1, 'splitted_texts_chunks'])
# print(x)

In [113]:

for i, row in df_exploded_2a.iterrows():

    print(i, '  --  ',row['splitted_texts_chunks'])
    
    emb = embeddings.embed_query(df_exploded_2a.loc[i, 'splitted_texts_chunks'])
    emb_val = ','.join(map(str, emb))
    df_exploded_2a.loc[i, 'splitted_texts_chunks_emb'] = emb_val
    
    # df_exploded_2a.loc[i, 'splitted_texts_chunks_emb_list'] = emb

    
df_exploded_2a['id'] = df_exploded_2a.index
df_exploded_2a = df_exploded_2a.rename(columns={'splitted_texts_chunks_emb': 'embedding'})


0   --   IN WHICH
1   --   We Are Introduced to
CHAPTER I
Winnie-the-Pooh and Some
Bees, and the Stories Begin
HERE is Edward B
2   --   ear, coming
downstairs now, bump, bump, bump, on the back
of his head, behind Christopher Robin. It 
3   --   is, as far
as he knows, the only way of coming downstairs,
but sometimes he feels that there really 
4   --   is another
way, if only he could stop bumping for a moment
and think of it. And then he feels that p
5   --   erhaps there
isn't. Anyhow, here he is at the bottom, and ready
to be introduced to you. Winnie-the-
6   --   Pooh.
When I first heard his name, I said, just as you
are going to say, "But I thought he was a boy
7   --   ?"
"So did I," said Christopher Robin.
"Then you can't call him Winnie?"
"I don't."
"But you said--"
8   --   
"He's Winnie-ther-Pooh. Don't you know what
'ther' means?”
I
Digitized by
Google

9   --   WINNIE-THE-POOH
“Ah, yes, now I do," I said quickly; and I hope
you do too, because it is all the ex
10   --   

In [415]:
df_exploded_2a[['id','splitted_texts_chunks','embedding']].head()

,id,splitted_texts_chunks,embedding
0,0,IN WHICH,"[0.008577119559049606, -0.012484382838010788, ..."
1,1,We Are Introduced to\nCHAPTER I\nWinnie-the-Po...,"[0.03151305764913559, -0.04322350397706032, -0..."
2,2,"ear, coming\ndownstairs now, bump, bump, bump,...","[-0.012076896615326405, -0.007126097101718187,..."
3,3,"is, as far\nas he knows, the only way of comin...","[-0.04710844159126282, 0.010321435518562794, -..."
4,4,"is another\nway, if only he could stop bumping...","[0.008820355869829655, 0.04707850515842438, 0...."


In [416]:
# save id and embedding as a json file
df_exploded_2a['id'] = df_exploded_2a['id'].astype(str)


jsonl_string = df_exploded_2a[["id",'splitted_texts_chunks',"embedding"]].to_json(orient="records", lines=True)
with open("questions.json", "w") as f:
    f.write(jsonl_string)

# show the first few lines of the json file
! head -n 3 questions.json

{"id":"0","splitted_texts_chunks":"IN WHICH","embedding":[0.0085771196,-0.0124843828,-0.0243481118,0.0374409072,-0.0323775746,-0.0078627206,0.0666385889,0.0651061982,-0.0060460684,-0.031819649,-0.0126534235,-0.0672832653,-0.0197192039,0.0101945894,0.0115754046,-0.0115177976,-0.0906385258,-0.0550257489,-0.000638693,0.0262025446,-0.0734190419,-0.0004111519,-0.0345363989,0.0427130871,-0.040865209,-0.0864942223,-0.0083778352,0.006187025,-0.0272276718,-0.0127419764,0.0207843762,-0.009607954,-0.0633966327,-0.0375701748,0.0475797839,0.0185279697,-0.0667250454,0.0181693882,0.0192275569,0.0141315581,0.0206093509,-0.025571892,0.0260585509,-0.0178616475,-0.0459052995,-0.0086374665,-0.0525466017,0.0067732041,-0.0105094155,-0.0422213748,-0.0317742303,-0.0097818859,-0.0235760063,0.0031715017,0.0452542342,0.0008397644,-0.0138043491,0.0144003844,0.0196846686,-0.0694785863,0.0094444379,0.0338444449,0.0448171794,-0.0636041686,-0.0173522159,0.0341138095,0.0479527228,-0.0178759359,-0.0062941257,-0.0218456

In [130]:
# import json

# # Assuming df_exploded_2a is a pandas DataFrame

# # Convert the "embedding" column to a list of floats
# df_exploded_2a["embedding"] = df_exploded_2a["embedding"].apply(lambda x: list(map(float, x.split(','))))

# # Convert DataFrame to JSON with the desired format
# jsonl_list = df_exploded_2a[["id","embedding"]].to_dict(orient="records")

# # Convert the list of dicts to the desired output format
# output_list = [{"id":str(item["id"]), "embedding":item["embedding"]} for item in jsonl_list]

# # Write the output to a JSON file
# with open("questions.json", "w") as f:
#     for item in output_list:
#         f.write(json.dumps(item) + "\n")
# ! head -n 3 questions.json

In [131]:
type(output_list)

list

In [132]:
# import json

# # Your input JSON string

# # with open("questions.json", "w") as f:
# #     for item in output_list:
# #         print(item)
# #         item_nospace = {key: value.replace(' ', '') if isinstance(value, str) else value for key, value in item.items()}
# #         f.write(json.dumps(item_nospace) + "\n")

# with open("questions.json", "w") as f:
#     for item in output_list:
#          embedding = item["embedding"].replace(' ', '')
#          f.write(json.dumps(question) + "\n")

# # print(json_string_no_spaces)
# # # Re-encode the dictionary to a formatted JSON string
# # formatted_json_string = json.dumps(data, separators=(',', ':'), indent=2)

# # print(formatted_json_string)
# # ! head -n 3 questions.json

In [133]:
! head -n 3 product-embs.json

{"id":"11863","name":"wacoal women's embrace lace chemise - 814191","embedding":[0.024530868977308273,-0.040793128311634064,0.00047576811630278826,0.017147192731499672,0.026629658415913582,-0.06141049787402153,0.020689481869339943,0.032609399408102036,-0.043242260813713074,-0.0030736858025193214,0.029859514907002449,-0.0070092626847326756,0.0074279718101024628,0.0044401995837688446,0.012789646163582802,0.0056940736249089241,-0.050746820867061615,-0.016834203153848648,-0.0025733353104442358,0.030782531946897507,-0.028840042650699615,0.013945236802101135,-0.0078231878578662872,-0.073326528072357178,-0.028099022805690765,-0.091063208878040314,0.016907420009374619,-0.0062174019403755665,0.0086391577497124672,-0.021324845030903816,-0.030394084751605988,-0.042661987245082855,-0.021932870149612427,-0.024862896651029587,0.025864196941256523,0.0095110638067126274,-0.023645631968975067,0.018030215054750443,0.013453914783895016,0.014843377284705639,-0.021920621395111084,0.0013795378617942333,-0.0

In [78]:
# # save id and embedding as a json file
# jsonl_string = df_exploded_2a[["id", "embedding"]].to_json(orient="records", lines=True)
# with open("questions.json", "w") as f:
#     f.write(jsonl_string)

# # show the first few lines of the json file
# ! head -n 3 questions.json

### Upload the Json File to matching engine

In [437]:
# define project information
if PROJECT_ID == "(unset)":
    PROJECT_ID = "[your-project-id]"  # @param {type:"string"}

# generate an unique id for this session
from datetime import datetime

UID = datetime.now().strftime("%m%d%H%M")

BUCKET_URI_ME="gs://my-project-0004-346516/matchingengine/embedding"
LOCATION = 'us-central1'

from google.cloud import aiplatform

aiplatform.init(project=PROJECT_ID, location=LOCATION)

In [438]:
! gsutil cp questions.json {BUCKET_URI_ME}


Copying file://questions.json [Content-Type=application/json]...
/ [1 files][244.5 KiB/244.5 KiB]                                                
Operation completed over 1 objects/244.5 KiB.                                    


In [439]:
BUCKET_URI_ME

'gs://my-project-0004-346516/matchingengine/embedding'

In [440]:

# create Index
my_index = aiplatform.MatchingEngineIndex.create_tree_ah_index(
    display_name=f"vs-feature-index-{UID}",
    contents_delta_uri=BUCKET_URI_ME,
    dimensions=768,
    approximate_neighbors_count=10,
    project = PROJECT_ID
)

Creating MatchingEngineIndex
Create MatchingEngineIndex backing LRO: projects/255766800726/locations/us-central1/indexes/1757428599511580672/operations/2186844996043276288
MatchingEngineIndex created. Resource name: projects/255766800726/locations/us-central1/indexes/1757428599511580672
To use this MatchingEngineIndex in another session:
index = aiplatform.MatchingEngineIndex('projects/255766800726/locations/us-central1/indexes/1757428599511580672')


#### Create Index Endpoint and deploy the Index
To use the Index, you need to create an Index Endpoint. It works as a server instance accepting query requests for your Index.


In [441]:
# create IndexEndpoint
my_index_endpoint = aiplatform.MatchingEngineIndexEndpoint.create(
    display_name=f"vs-feature-index-endpoint-{UID}", public_endpoint_enabled=True
)

Creating MatchingEngineIndexEndpoint
Create MatchingEngineIndexEndpoint backing LRO: projects/255766800726/locations/us-central1/indexEndpoints/6265250351532736512/operations/5194123647219924992
MatchingEngineIndexEndpoint created. Resource name: projects/255766800726/locations/us-central1/indexEndpoints/6265250351532736512
To use this MatchingEngineIndexEndpoint in another session:
index_endpoint = aiplatform.MatchingEngineIndexEndpoint('projects/255766800726/locations/us-central1/indexEndpoints/6265250351532736512')


In [ ]:
DEPLOYED_INDEX_ID = f"vs_feature_deployed_{UID}"
# deploy the Index to the Index Endpoint
my_index_endpoint.deploy_index(index=my_index, deployed_index_id=DEPLOYED_INDEX_ID)

Deploying index MatchingEngineIndexEndpoint index_endpoint: projects/255766800726/locations/us-central1/indexEndpoints/6265250351532736512
Deploy index MatchingEngineIndexEndpoint index_endpoint backing LRO: projects/255766800726/locations/us-central1/indexEndpoints/6265250351532736512/operations/1426862558924505088


### Github matching engine deployment Test

In [92]:
! gsutil cp "gs://github-repo/data/vs-quickstart/product-embs.json" "$BUCKET_URI_ME"


Copying gs://github-repo/data/vs-quickstart/product-embs.json [Content-Type=application/json]...
- [1 files][ 79.3 MiB/ 79.3 MiB]                                                
Operation completed over 1 objects/79.3 MiB.                                     


In [512]:
! gsutil cp "gs://github-repo/data/vs-quickstart/product-embs.json" . # for query tests


Copying gs://github-repo/data/vs-quickstart/product-embs.json...
\ [1 files][ 79.3 MiB/ 79.3 MiB]                                                
Operation completed over 1 objects/79.3 MiB.                                     


In [516]:
# create Index
my_index = aiplatform.MatchingEngineIndex.create_tree_ah_index(
    display_name=f"vs-test-index-{UID}",
    contents_delta_uri=BUCKET_URI,
    dimensions=768,
    approximate_neighbors_count=10,
    project = PROJECT_ID
)

Creating MatchingEngineIndex
Create MatchingEngineIndex backing LRO: projects/801452371447/locations/us-central1/indexes/3567875649714520064/operations/9173440946094735360


KeyboardInterrupt: 

In [514]:
# create IndexEndpoint
my_index_endpoint = aiplatform.MatchingEngineIndexEndpoint.create(
    display_name=f"vs-test-index-endpoint-{UID}", public_endpoint_enabled=True
)

FailedPrecondition: 400 Exceeds maximum number of PublicEndpoint(s) allowed for a single Project, which is 15.

In [515]:
!gcloud projects list

PROJECT_ID             NAME                   PROJECT_NUMBER
cloud-llm-preview1     cloud-llm-preview1     801452371447
cloud-tpu-pubsubs      cloud-tpu-pubsubs      190395505491
cloud-tpu-v2-images    cloud-tpu-v2-images    202321179179
psprod-asia-east1-c    psprod-asia-east1-c    745274831519
psprod-europe-west4-a  psprod-europe-west4-a  632574482707
psprod-europe-west4-b  psprod-europe-west4-b  453095801626
psprod-us-central1-a   psprod-us-central1-a   577258445108
psprod-us-central1-b   psprod-us-central1-b   250731689161
psprod-us-central1-c   psprod-us-central1-c   991251160362
psprod-us-central1-f   psprod-us-central1-f   1010488386695
psprod-us-central2-b   psprod-us-central2-b   751422327981
psprod-us-east1-c      psprod-us-east1-c      136293625253
psprod-us-east1-d      psprod-us-east1-d      769222551881
psprod-us-east5-a      psprod-us-east5-a      624264038575
psprod-us-east5-b      psprod-us-east5-b      359592342441
psprod-us-west1-b      psprod-us-west1-b      5965611

### Deploy instead using langchiain

In [558]:
PROJECT_ID = "my-project-0004-346516"
REGION = "us-central1"
VPC_NETWORK = "matching-engine-vpc"
PEERING_RANGE_NAME = "ann-langchain-me-range"  # Name for creating the VPC peering.
# BUCKET_URI = "gs://<bucket_uri>"
BUCKET_URI="gs://tianhaoz-test"

# The number of dimensions for the tensorflow universal sentence encoder its 500
# If other embedder is used, the dimensions would probably need to change. for gecko its 768
DIMENSIONS = 768
DISPLAY_NAME = "index-test-name-new"
EMBEDDING_DIR = f"{BUCKET_URI}/saurabh"
DEPLOYED_INDEX_ID = "endpoint-test-name"

PROJECT_NUMBER = !gcloud projects list --filter="PROJECT_ID:'{PROJECT_ID}'" --format='value(PROJECT_NUMBER)'
PROJECT_NUMBER = PROJECT_NUMBER[0]
VPC_NETWORK_FULL = f"projects/{PROJECT_NUMBER}/global/networks/{VPC_NETWORK}"

CREATE_VPC = True

In [559]:
# # Remove the if condition to run the encapsulated code
# if CREATE_VPC:
#     # Create a VPC network
#     ! gcloud compute networks create {VPC_NETWORK} --bgp-routing-mode=regional --subnet-mode=auto --project={PROJECT_ID}

#     # Add necessary firewall rules
#     ! gcloud compute firewall-rules create {VPC_NETWORK}-allow-icmp --network {VPC_NETWORK} --priority 65534 --project {PROJECT_ID} --allow icmp

#     ! gcloud compute firewall-rules create {VPC_NETWORK}-allow-internal --network {VPC_NETWORK} --priority 65534 --project {PROJECT_ID} --allow all --source-ranges 10.128.0.0/9

#     ! gcloud compute firewall-rules create {VPC_NETWORK}-allow-rdp --network {VPC_NETWORK} --priority 65534 --project {PROJECT_ID} --allow tcp:3389

#     ! gcloud compute firewall-rules create {VPC_NETWORK}-allow-ssh --network {VPC_NETWORK} --priority 65534 --project {PROJECT_ID} --allow tcp:22

#     # Reserve IP range
#     ! gcloud compute addresses create {PEERING_RANGE_NAME} --global --prefix-length=16 --network={VPC_NETWORK} --purpose=VPC_PEERING --project={PROJECT_ID} --description="peering range"

#     # Set up peering with service networking
#     # Your account must have the "Compute Network Admin" role to run the following.
#     ! gcloud services vpc-peerings connect --service=servicenetworking.googleapis.com --network={VPC_NETWORK} --ranges={PEERING_RANGE_NAME} --project={PROJECT_ID}

In [560]:
my_index = aiplatform.MatchingEngineIndex.create_brute_force_index(
    display_name=DISPLAY_NAME,
    contents_delta_uri=EMBEDDING_DIR,
    dimensions=DIMENSIONS,
    # approximate_neighbors_count=150,
    distance_measure_type="DOT_PRODUCT_DISTANCE",
)

Creating MatchingEngineIndex
Create MatchingEngineIndex backing LRO: projects/801452371447/locations/us-central1/indexes/280810871687479296/operations/5901294341833359360
MatchingEngineIndex created. Resource name: projects/801452371447/locations/us-central1/indexes/280810871687479296
To use this MatchingEngineIndex in another session:
index = aiplatform.MatchingEngineIndex('projects/801452371447/locations/us-central1/indexes/280810871687479296')


In [561]:
my_index = aiplatform.MatchingEngineIndex('projects/801452371447/locations/us-central1/indexes/280810871687479296')
